<a href="https://colab.research.google.com/github/Eskarteek/Eskarteek_DATA606/blob/main/EDA/Capstone_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to my Notebook

### importing libraries and packages


In [70]:
# pip install nltk

In [72]:
# pip install snscrape

In [74]:
# pip install yfinance

In [76]:
# nltk.download('vader_lexicon')

In [78]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [85]:
# pip install drive

In [157]:
import pandas as pd
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import nltk
import numpy as np
from bs4 import BeautifulSoup
import requests
import snscrape.modules.twitter as sntwitter
import datetime

In [89]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [178]:
#picking stock symbol
data = pd.read_csv('/content/gdrive/MyDrive/nasdaq_screener_1645401938081.csv')
def User_inputs():
  symbol = input("Enter the stock symbol: ").upper()
  while True:
    if symbol not in data['Symbol'].to_list():
      print('Symbol not available in our database or you have entered it wrong')
      symbol = input("Enter the stock symbol: ").upper()
    else:
      return symbol

In [179]:
#Reading NASDAQ data
symbol = User_inputs()

Enter the stock symbol: AMZN


In [180]:
symbol

'AMZN'

In [144]:
data_dict = {}

In [147]:
for i in range(len(data)):
  data_dict[data.iloc[i][0]] = [data.iloc[i][1], data.iloc[i][2]]

In [262]:
data['Name'][6].split('.')

['Applied Optoelectronics Inc', ' Common Stock']

In [114]:
#Stock symbol and company name data.
# data_dict

In [115]:
import yfinance as yf

In [116]:
#Scrapes all the news articles from yfinance website
inp = yf.Ticker(symbol)

In [1]:
inp.news

NameError: ignored

In [118]:
#Scraping text from news articles and titles using beautiful soup
df = pd.DataFrame()
title = []
newslist = []
for i in range(len(inp.news)):
  site = inp.news[i]['link']
  r = requests.get(site)
  soup = BeautifulSoup(r.text)
  try:
    newslist.append(soup.find('div', attrs={'class': 'caas-body'}).text)
    title.append(inp.news[i]['title'])
  except:
    pass
  

In [119]:
df['Title'] = title

In [120]:
df['News'] = newslist

In [121]:
df

,Title,News
0,Millennial And Gen Z Stock Picks From Tesla To...,Millennials and Gen Z are investing earlier th...
1,Is Lucid Motors Stock A Buy Right Now After Q4...,"Lucid Motors stock debuted in late July, surgi..."
2,Berkshire Hathaway’s Market Cap Climbs. It Cou...,Value stocks are getting more love than growth...
3,"Biden elevates Ford, GM, and now Siemens — but...",When President Joe Biden wanted to talk about ...
4,These Are The 5 Best EV Stocks To Buy And Watc...,EV stocks have multiplied in Tesla’s wake and ...


In [122]:
# Using VADAR sentiment analyzer to detected sentiment scores in the text
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [123]:
vader = SentimentIntensityAnalyzer()
scores = df['News'].apply(vader.polarity_scores).tolist()
df['Score'] = scores
#Adding these scores to the dataset

In [124]:
df

,Title,News,Score
0,Millennial And Gen Z Stock Picks From Tesla To...,Millennials and Gen Z are investing earlier th...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp..."
1,Is Lucid Motors Stock A Buy Right Now After Q4...,"Lucid Motors stock debuted in late July, surgi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,Berkshire Hathaway’s Market Cap Climbs. It Cou...,Value stocks are getting more love than growth...,"{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'comp..."
3,"Biden elevates Ford, GM, and now Siemens — but...",When President Joe Biden wanted to talk about ...,"{'neg': 0.01, 'neu': 0.937, 'pos': 0.054, 'com..."
4,These Are The 5 Best EV Stocks To Buy And Watc...,EV stocks have multiplied in Tesla’s wake and ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [125]:
#Predicting Polarity (0: Neutal , Positive: 1,Negative: -1)
pn = []
for i in df['Score']:
    if i['compound'] > 0:    
        pn.append(1)
    elif i['compound'] < 0:
        pn.append(-1)
    else:
      pn.append(0)

In [126]:
#Appending them to the dataset
df['Sentiment Poles'] = pn

In [127]:
df

,Title,News,Score,Sentiment Poles
0,Millennial And Gen Z Stock Picks From Tesla To...,Millennials and Gen Z are investing earlier th...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp...",1
1,Is Lucid Motors Stock A Buy Right Now After Q4...,"Lucid Motors stock debuted in late July, surgi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0
2,Berkshire Hathaway’s Market Cap Climbs. It Cou...,Value stocks are getting more love than growth...,"{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'comp...",1
3,"Biden elevates Ford, GM, and now Siemens — but...",When President Joe Biden wanted to talk about ...,"{'neg': 0.01, 'neu': 0.937, 'pos': 0.054, 'com...",1
4,These Are The 5 Best EV Stocks To Buy And Watc...,EV stocks have multiplied in Tesla’s wake and ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0


In [128]:
# Data cleaning
sw = nltk.corpus.stopwords.words("english")
lemma = nltk.stem.wordnet.WordNetLemmatizer()

In [43]:
# #Visualizing using wordcloud and finding key words in the text
# for text in df['News']:
#   print(text)
#   words = text.split(' ')
#   words = [word for word in words if word not in sw]
#   words = [lemma.lemmatize(word) for word in words]
#   fdist1 = nltk.FreqDist(words)
#   plt.figure(figsize = (10,6))
#   wordcloud = WordCloud().generate(text)
#   plt.imshow(wordcloud, interpolation='bilinear')
#   plt.axis("off")
#   plt.show()
#   print (fdist1.most_common(20))

In [40]:
# pip install streamlit

In [41]:
# import streamlit as st

In [42]:
# st.write(""" # My First App 
#             Hello *world!* """ )

In [245]:
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=100)

In [246]:
t_day = str(today.year)+'-'+str(today.month)+'-'+str(today.day)

In [247]:
y_day = str(yesterday.year)+'-'+str(yesterday.month)+'-'+str(yesterday.day)

In [248]:
string = f'{data_dict[symbol][0]} since:{y_day} until:{t_day}'

In [249]:
string

'Amazon.com Inc. Common Stock since:2021-11-27 until:2022-3-7'

In [250]:
import snscrape

In [251]:
import snscrape.modules.twitter as sntwitter
import pandas

# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(string).get_items()):
    if i>5000:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
    
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])


In [252]:
print(tweets_df2['Text'])

0    $VLDR $AMZN - Velodyne Lidar, Inc.  and Amazon...
1    ですよね。\n&gt;Fourth quarter 2021 net income incl...
2    (12/23) SEATTLE, WA (May 14, 1997) - https://t...
3    #AMZN https://t.co/WtuoBmDK2k, Inc. #Common #S...
Name: Text, dtype: object


In [253]:
print(f'{data_dict[symbol][0]}:{y_day} until:{t_day}')

Amazon.com Inc. Common Stock:2021-11-27 until:2022-3-7


In [254]:
tweets_df2

,Datetime,Tweet Id,Text,Username
0,2022-02-07 21:46:09+00:00,1490804099239002120,"$VLDR $AMZN - Velodyne Lidar, Inc. and Amazon...",HammerstoneMar3
1,2022-02-03 21:37:06+00:00,1489352267526389762,ですよね。\n&gt;Fourth quarter 2021 net income incl...,nagataka
2,2022-01-12 16:04:51+00:00,1481296124648505347,"(12/23) SEATTLE, WA (May 14, 1997) - https://t...",finsily
3,2021-12-14 20:35:03+00:00,1470854874837524486,"#AMZN https://t.co/WtuoBmDK2k, Inc. #Common #S...",prudent_charly
